Upon data study, this notebook will create new columns from flight delay claim dataset, that could be used as features in the prediction model.

In [178]:
# Used libraries
import pandas as pd
from datetime import datetime, timedelta

In [179]:
# Load the dataset
data_df = pd.read_csv('../datasets/flight_delays_data.csv')

# Check data size
data_df.shape

(899114, 10)

In [180]:
# Show some sample data
data_df.head()

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0
2,1582504,PR301,14,HKG,MNL,PR,11,0.0,2014-04-08,0
3,1582508,LD327,37,HKG,SIN,LD,3,0.1,2013-09-15,0
4,1582509,KA5390,40,HKG,PEK,KA,9,0.5,2015-10-05,0


From the study, we are going to create different types of statistics that help with the prediction:

# Delay hours statistics

From the study, we are going to create statistics of delay hours from different perspectives:
- Departure
- Arrival
- Airline

Below are not tested now first ...
- Departure + Arrival
- Departure + Airline (i.e. ~= Airline in this dataset)
- Arrival + Airline
- Departure + Arrival + Airline

For each perspective, the average delay_time of per (1 hour/1 day/1 week) is computed. After that, deviation of consecutive last average delay_time is computed.

In [181]:
# Get flight datetime-related columns
data_df['flight_date_dt'] = data_df['flight_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
data_df['flight_dt'] = data_df.apply(lambda x: x['flight_date_dt'] + timedelta(hours=x['std_hour']), axis=1)
data_df['flight_year'] = data_df['flight_dt'].apply(lambda x: x.year)
data_df['flight_month'] = data_df['flight_dt'].apply(lambda x: x.month)
data_df['flight_day'] = data_df['flight_dt'].apply(lambda x: x.day)
data_df['flight_hour_bin'] = data_df['std_hour'].apply(lambda x: x // 4)

In [182]:
data_df.sample(10)

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,flight_date_dt,flight_dt,flight_year,flight_month,flight_day,flight_hour_bin
353008,1098608,UO192,44,HKG,RMQ,UO,18,0.5,2015-10-31,0,2015-10-31,2015-10-31 18:00:00,2015,10,31,4
678148,2108038,CX1729,36,HKG,KUL,CX,20,0.2,2014-09-05,0,2014-09-05,2014-09-05 20:00:00,2014,9,5,5
409303,1273685,CX711,6,HKG,SIN,CX,16,0.1,2015-02-11,0,2015-02-11,2015-02-11 16:00:00,2015,2,11,4
429791,1336860,BA7410,46,HKG,MEL,BA,19,0.3,2013-11-14,0,2013-11-14,2013-11-14 19:00:00,2013,11,14,4
269528,838468,CX6505,19,HKG,USM,CX,9,0.4,2016-05-07,0,2016-05-07,2016-05-07 09:00:00,2016,5,7,2
601369,1870487,CX5950,42,HKG,TAO,CX,10,0.8,2015-10-17,0,2015-10-17,2015-10-17 10:00:00,2015,10,17,2
766665,2381724,CX271,13,HKG,AMS,CX,0,0.0,2015-03-30,0,2015-03-30,2015-03-30 00:00:00,2015,3,30,0
425022,1322050,CX697,46,HKG,DEL,CX,20,0.2,2014-11-14,0,2014-11-14,2014-11-14 20:00:00,2014,11,14,5
850527,2642174,CX751,3,HKG,BKK,CX,14,0.1,2015-01-19,0,2015-01-19,2015-01-19 14:00:00,2015,1,19,3
721891,2243250,CX111,25,HKG,SYD,CX,18,0.2,2015-06-20,0,2015-06-20,2015-06-20 18:00:00,2015,6,20,4


In [183]:
# Before processing delay_time statistics, remove cancalled entries first
non_cancel_df = data_df[~(data_df['delay_time'] == "Cancelled")].copy()

# Translate delay_time to float
non_cancel_df['delay_time'] = non_cancel_df['delay_time'].apply(lambda x: float(x))

# Note: groupby preserves the order of rows within each group
non_cancel_df = non_cancel_df.sort_values(['Departure', 'Arrival', 'Airline', 'flight_year', 'flight_month', 'flight_day', 'std_hour'],ascending=False)

In [184]:
# Departure + hour
dep_hr_delay_df = non_cancel_df.groupby(['Departure', 'flight_year', 'flight_month', 'flight_day', 'std_hour']).mean()['delay_time'].reset_index()

In [185]:
# Arrival + hour
arr_hr_delay_df = non_cancel_df.groupby(['Arrival', 'flight_year', 'flight_month', 'flight_day', 'std_hour']).mean()['delay_time'].reset_index()

In [186]:
# Airline + hour
air_hr_delay_df = non_cancel_df.groupby(['Airline', 'flight_year', 'flight_month', 'flight_day', 'std_hour']).mean()['delay_time'].reset_index()

In [189]:
# For each stat df, create a time series key such that it is easier to select "most recent" value in later stage
# This key will not become a feature in model training
def get_time_series_val(row):
    flight_year = row['flight_year']
    flight_month = row['flight_month']
    flight_day = row['flight_day']
    std_hour = row['std_hour']
    return flight_year * (10 ** 6) + flight_month * (10 ** 4) + flight_day * (10 ** 2) + std_hour

dep_hr_delay_df['flight_ts'] = dep_hr_delay_df.apply(get_time_series_val, axis=1)
arr_hr_delay_df['flight_ts'] = arr_hr_delay_df.apply(get_time_series_val, axis=1)
air_hr_delay_df['flight_ts'] = air_hr_delay_df.apply(get_time_series_val, axis=1)

In [190]:
dep_hr_delay_df.rename(columns={'delay_time': 'dep_hr_delay_time'}, inplace=True)
arr_hr_delay_df.rename(columns={'delay_time': 'arr_hr_delay_time'}, inplace=True)
air_hr_delay_df.rename(columns={'delay_time': 'air_hr_delay_time'}, inplace=True)

In [191]:
dep_hr_delay_df

,Departure,flight_year,flight_month,flight_day,std_hour,dep_hr_delay_time,flight_ts
0,HKG,2013,9,1,0,0.205263,2013090100
1,HKG,2013,9,1,1,0.025000,2013090101
2,HKG,2013,9,1,2,-0.133333,2013090102
3,HKG,2013,9,1,3,0.320000,2013090103
4,HKG,2013,9,1,4,1.066667,2013090104
5,HKG,2013,9,1,5,0.400000,2013090105
6,HKG,2013,9,1,7,0.055556,2013090107
7,HKG,2013,9,1,8,0.254386,2013090108
8,HKG,2013,9,1,9,0.104348,2013090109
9,HKG,2013,9,1,10,0.067347,2013090110


# Cancelled status statistics

In [192]:
# Before processing delay_time statistics, remove cancalled entries first
cancel_df = data_df[(data_df['delay_time'] == "Cancelled")].copy()

# Translate delay_time to float
cancel_df['is_cancel'] = cancel_df['delay_time'].apply(lambda x: x == "Cancelled")

# Note: groupby preserves the order of rows within each group
cancel_df = cancel_df.sort_values(['Departure', 'Arrival', 'Airline', 'flight_year', 'flight_month', 'flight_day', 'std_hour'],ascending=False)

In [193]:
cancel_df

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,flight_date_dt,flight_dt,flight_year,flight_month,flight_day,flight_hour_bin,is_cancel
806612,2505450,HX2206,13,HKG,ZYI,HX,17,Cancelled,2016-03-28,800,2016-03-28,2016-03-28 17:00:00,2016,3,28,4,True
566525,1762897,HX2206,9,HKG,ZYI,HX,17,Cancelled,2016-02-29,800,2016-02-29,2016-02-29 17:00:00,2016,2,29,4,True
149160,462548,HX2206,9,HKG,ZYI,HX,17,Cancelled,2016-02-26,800,2016-02-26,2016-02-26 17:00:00,2016,2,26,4,True
752647,2338383,HX2206,8,HKG,ZYI,HX,17,Cancelled,2016-02-19,800,2016-02-19,2016-02-19 17:00:00,2016,2,19,4,True
608406,1892235,HX2206,7,HKG,ZYI,HX,17,Cancelled,2016-02-12,800,2016-02-12,2016-02-12 17:00:00,2016,2,12,4,True
885495,2750955,HX2206,6,HKG,ZYI,HX,17,Cancelled,2016-02-08,800,2016-02-08,2016-02-08 17:00:00,2016,2,8,4,True
740698,2301325,HX2206,6,HKG,ZYI,HX,17,Cancelled,2016-02-05,800,2016-02-05,2016-02-05 17:00:00,2016,2,5,4,True
157370,487932,HX2206,5,HKG,ZYI,HX,17,Cancelled,2016-01-29,800,2016-01-29,2016-01-29 17:00:00,2016,1,29,4,True
631437,1963355,HX2206,4,HKG,ZYI,HX,17,Cancelled,2016-01-25,800,2016-01-25,2016-01-25 17:00:00,2016,1,25,4,True
526252,1637839,HX2206,4,HKG,ZYI,HX,17,Cancelled,2016-01-22,800,2016-01-22,2016-01-22 17:00:00,2016,1,22,4,True


In [194]:
# Departure + hour
dep_hr_cancel_df = cancel_df.groupby(['Departure', 'flight_year', 'flight_month', 'flight_day', 'std_hour']).count()['is_cancel'].reset_index()

In [195]:
# Arrival + hour
arr_hr_cancel_df = cancel_df.groupby(['Arrival', 'flight_year', 'flight_month', 'flight_day', 'std_hour']).count()['is_cancel'].reset_index()

In [196]:
# Airline + hour
air_hr_cancel_df = cancel_df.groupby(['Airline', 'flight_year', 'flight_month', 'flight_day', 'std_hour']).count()['is_cancel'].reset_index()

In [197]:
# For each stat df, create a time series key such that it is easier to select "most recent" value in later stage
# This key will not become a feature in model training
def get_time_series_val(row):
    flight_year = row['flight_year']
    flight_month = row['flight_month']
    flight_day = row['flight_day']
    std_hour = row['std_hour']
    return flight_year * (10 ** 6) + flight_month * (10 ** 4) + flight_day * (10 ** 2) + std_hour

dep_hr_cancel_df['flight_ts'] = dep_hr_cancel_df.apply(get_time_series_val, axis=1)
arr_hr_cancel_df['flight_ts'] = arr_hr_cancel_df.apply(get_time_series_val, axis=1)
air_hr_cancel_df['flight_ts'] = air_hr_cancel_df.apply(get_time_series_val, axis=1)

In [198]:
dep_hr_cancel_df.rename(columns={'is_cancel': 'dep_hr_cancel_count'}, inplace=True)
arr_hr_cancel_df.rename(columns={'is_cancel': 'arr_hr_cancel_count'}, inplace=True)
air_hr_cancel_df.rename(columns={'is_cancel': 'air_hr_cancel_count'}, inplace=True)

In [199]:
dep_hr_cancel_df

,Departure,flight_year,flight_month,flight_day,std_hour,dep_hr_cancel_count,flight_ts
0,HKG,2013,9,1,0,1,2013090100
1,HKG,2013,9,1,9,1,2013090109
2,HKG,2013,9,1,10,4,2013090110
3,HKG,2013,9,1,11,7,2013090111
4,HKG,2013,9,1,14,1,2013090114
5,HKG,2013,9,1,19,1,2013090119
6,HKG,2013,9,1,21,3,2013090121
7,HKG,2013,9,1,22,1,2013090122
8,HKG,2013,9,2,9,4,2013090209
9,HKG,2013,9,2,10,2,2013090210


Merging the statistics to original dataset ...

In [211]:
def get_last_time_series_val(row):
    flight_dt = row['flight_dt'] - timedelta(hours=1)
    return int(flight_dt.strftime("%Y%m%d%H"))

In [212]:
feature_df = data_df.copy()
feature_df['flight_ts'] = feature_df.apply(get_last_time_series_val, axis=1)

In [200]:
feature_df

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,flight_date_dt,flight_dt,flight_year,flight_month,flight_day,flight_hour_bin,flight_ts
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0,2016-07-01,2016-07-01 10:00:00,2016,7,1,2,2016070109
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0,2015-04-23,2015-04-23 11:00:00,2015,4,23,2,2015042310
2,1582504,PR301,14,HKG,MNL,PR,11,0.0,2014-04-08,0,2014-04-08,2014-04-08 11:00:00,2014,4,8,2,2014040810
3,1582508,LD327,37,HKG,SIN,LD,3,0.1,2013-09-15,0,2013-09-15,2013-09-15 03:00:00,2013,9,15,0,2013091502
4,1582509,KA5390,40,HKG,PEK,KA,9,0.5,2015-10-05,0,2015-10-05,2015-10-05 09:00:00,2015,10,5,2,2015100508
5,1582511,NZ4851,10,HKG,IST,NZ,23,0.2,2015-03-10,0,2015-03-10,2015-03-10 23:00:00,2015,3,10,5,2015031022
6,1582512,CX5626,51,HKG,HGH,CX,7,0.4,2015-12-19,0,2015-12-19,2015-12-19 07:00:00,2015,12,19,1,2015121906
7,1582513,MH9725,8,HKG,KUL,MH,12,0.1,2014-02-23,0,2014-02-23,2014-02-23 12:00:00,2014,2,23,3,2014022311
8,1582516,KA154,42,HKG,BLR,KA,21,0.0,2014-10-21,0,2014-10-21,2014-10-21 21:00:00,2014,10,21,5,2014102120
9,1582517,CX233,10,HKG,MXP,CX,0,0.3,2015-03-05,0,2015-03-05,2015-03-05 00:00:00,2015,3,5,0,2015030423


In [213]:
# TODO: Use last time rather than current time
# Just to test for the idea first
merged_feature_df = feature_df.merge(dep_hr_delay_df[['Departure', 'flight_ts', 'dep_hr_delay_time']], how='left', left_on=['Departure', 'flight_ts'], right_on=['Departure', 'flight_ts'])
merged_feature_df = merged_feature_df.merge(arr_hr_delay_df[['Arrival', 'flight_ts', 'arr_hr_delay_time']], how='left', left_on=['Arrival', 'flight_ts'], right_on=['Arrival', 'flight_ts'])
merged_feature_df = merged_feature_df.merge(air_hr_delay_df[['Airline', 'flight_ts', 'air_hr_delay_time']], how='left', left_on=['Airline', 'flight_ts'], right_on=['Airline', 'flight_ts'])

In [216]:
merged_feature_df = merged_feature_df.merge(dep_hr_cancel_df[['Departure', 'flight_ts', 'dep_hr_cancel_count']], how='left', left_on=['Departure', 'flight_ts'], right_on=['Departure', 'flight_ts'])
merged_feature_df = merged_feature_df.merge(arr_hr_cancel_df[['Arrival', 'flight_ts', 'arr_hr_cancel_count']], how='left', left_on=['Arrival', 'flight_ts'], right_on=['Arrival', 'flight_ts'])
merged_feature_df = merged_feature_df.merge(air_hr_cancel_df[['Airline', 'flight_ts', 'air_hr_cancel_count']], how='left', left_on=['Airline', 'flight_ts'], right_on=['Airline', 'flight_ts'])

In [217]:
merged_feature_df['is_claim'].value_counts()

0      859701
800     39413
Name: is_claim, dtype: int64

In [218]:
merged_feature_df.dropna()['is_claim'].value_counts()

0      3536
800     579
Name: is_claim, dtype: int64

In [219]:
merged_feature_df.to_csv('../datasets/flight_delays_data_transformed.csv')

In [157]:
merged_feature_df.fillna(0)

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,...,flight_month,flight_day,flight_hour_bin,flight_ts,dep_hr_delay_time,arr_hr_delay_time,air_hr_delay_time,dep_hr_cancel_count,arr_hr_cancel_count,air_hr_cancel_count
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0,...,7,1,2,2016070110,0.483673,0.200000,0.300000,0.0,0.0,0.0
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0,...,4,23,2,2015042311,0.238889,0.500000,0.266667,2.0,0.0,0.0
2,1582504,PR301,14,HKG,MNL,PR,11,0.0,2014-04-08,0,...,4,8,2,2014040811,0.216216,0.000000,0.000000,0.0,0.0,0.0
3,1582508,LD327,37,HKG,SIN,LD,3,0.1,2013-09-15,0,...,9,15,0,2013091503,0.125000,0.100000,0.125000,0.0,0.0,0.0
4,1582509,KA5390,40,HKG,PEK,KA,9,0.5,2015-10-05,0,...,10,5,2,2015100509,1.359322,0.500000,1.366667,0.0,0.0,0.0
5,1582511,NZ4851,10,HKG,IST,NZ,23,0.2,2015-03-10,0,...,3,10,5,2015031023,0.276471,0.200000,0.150000,3.0,0.0,0.0
6,1582512,CX5626,51,HKG,HGH,CX,7,0.4,2015-12-19,0,...,12,19,1,2015121907,0.520000,0.400000,0.580000,0.0,0.0,0.0
7,1582513,MH9725,8,HKG,KUL,MH,12,0.1,2014-02-23,0,...,2,23,3,2014022312,0.106977,0.100000,0.100000,3.0,0.0,0.0
8,1582516,KA154,42,HKG,BLR,KA,21,0.0,2014-10-21,0,...,10,21,5,2014102121,0.268571,0.000000,0.100000,0.0,0.0,0.0
9,1582517,CX233,10,HKG,MXP,CX,0,0.3,2015-03-05,0,...,3,5,0,2015030500,0.117647,0.300000,0.116667,0.0,0.0,0.0


In [86]:
feature_df

,flight_id,flight_no,Week,Departure,Arrival,Airline,std_hour,delay_time,flight_date,is_claim,flight_date_dt,flight_dt,flight_year,flight_month,flight_day,flight_hour_bin
0,1582499,UO686,27,HKG,KIX,UO,10,0.4,2016-07-01,0,2016-07-01,2016-07-01 10:00:00,2016,7,1,2
1,1582501,CI7868,17,HKG,TNN,CI,11,0.5,2015-04-23,0,2015-04-23,2015-04-23 11:00:00,2015,4,23,2
2,1582504,PR301,14,HKG,MNL,PR,11,0.0,2014-04-08,0,2014-04-08,2014-04-08 11:00:00,2014,4,8,2
3,1582508,LD327,37,HKG,SIN,LD,3,0.1,2013-09-15,0,2013-09-15,2013-09-15 03:00:00,2013,9,15,0
4,1582509,KA5390,40,HKG,PEK,KA,9,0.5,2015-10-05,0,2015-10-05,2015-10-05 09:00:00,2015,10,5,2
5,1582511,NZ4851,10,HKG,IST,NZ,23,0.2,2015-03-10,0,2015-03-10,2015-03-10 23:00:00,2015,3,10,5
6,1582512,CX5626,51,HKG,HGH,CX,7,0.4,2015-12-19,0,2015-12-19,2015-12-19 07:00:00,2015,12,19,1
7,1582513,MH9725,8,HKG,KUL,MH,12,0.1,2014-02-23,0,2014-02-23,2014-02-23 12:00:00,2014,2,23,3
8,1582516,KA154,42,HKG,BLR,KA,21,0.0,2014-10-21,0,2014-10-21,2014-10-21 21:00:00,2014,10,21,5
9,1582517,CX233,10,HKG,MXP,CX,0,0.3,2015-03-05,0,2015-03-05,2015-03-05 00:00:00,2015,3,5,0


In [82]:
air_hr_delay_df[(air_hr_delay_df['Airline'] == 'ZH') & (air_hr_delay_df['flight_ts'] < 2016072416)].iloc[-1].delay_time

0.80000000000000004

In [87]:
def get_last_dep_hr_delay_time(row):
    flight_year = row['flight_year']
    flight_month = row['flight_month']
    flight_day = row['flight_day']
    std_hour = row['std_hour']
    dep = row['Departure']
    flight_ts = flight_year * (10 ** 6) + flight_month * (10 ** 4) + flight_day * (10 ** 2) + std_hour
    
    last_dep_hr_df = dep_hr_delay_df[(dep_hr_delay_df['Departure'] == 'dep') & (dep_hr_delay_df['flight_ts'] < flight_ts)]
    if len(last_dep_hr_df) > 0:
        selected_row = last_dep_hr_df.iloc[-1]
        return selected_row.delay_time
    else:
        return None
    
feature_df['last_dep_hr_delay_time'] = feature_df.apply(get_last_dep_hr_delay_time, axis=1)

KeyboardInterrupt: 

In [44]:
# Departure + day
dep_day_delay_df = non_cancel_df.groupby(['Departure', 'flight_year', 'flight_month', 'flight_day']).mean()['delay_time'].reset_index()

In [ ]:
# Arrival + hour
arr_day_delay_df = non_cancel_df.groupby(['Arrival', 'flight_year', 'flight_month', 'flight_day', 'std_hour']).mean()['delay_time'].reset_index()

In [40]:
test_df[(test_df['flight_month'] == 9) & (test_df['flight_day'] > 21)]

,Departure,flight_year,flight_month,flight_day,std_hour,flight_id,Week,delay_time,is_claim,flight_hour_bin
459,HKG,2013,9,22,0,1.245541e+06,38.0,0.188889,0.000000,0.0
460,HKG,2013,9,22,1,1.271148e+06,38.0,0.065000,0.000000,0.0
461,HKG,2013,9,22,2,1.228075e+06,38.0,0.033333,0.000000,0.0
462,HKG,2013,9,22,3,1.437141e+06,38.0,0.000000,0.000000,0.0
463,HKG,2013,9,22,4,1.479844e+06,38.0,-0.080000,0.000000,1.0
464,HKG,2013,9,22,5,7.759900e+05,38.0,0.100000,0.000000,1.0
465,HKG,2013,9,22,7,1.418436e+06,38.0,0.023077,0.000000,1.0
466,HKG,2013,9,22,8,1.451535e+06,38.0,0.132812,0.000000,2.0
467,HKG,2013,9,22,9,1.408648e+06,38.0,0.423256,0.000000,2.0
468,HKG,2013,9,22,10,1.380346e+06,38.0,0.360870,0.000000,2.0


In addition, for each flight record, the rolling average of last (4 hours/1 day/1 week) is computed.

# Cancel status statistics